In [ ]:
import alpha_project
from alpha_project.Tree import Node, Tree
from alpha_project.LpuIndex import LpuIndex
from alpha_project.LpuList import LpuList
from alpha_project.Train import Train

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# import warnings
# warnings.simplefilter('ignore')

import os
import re
import time
import json
import nltk
import string
import collections
import requests
import numpy as np
import pandas as pd

from collections import OrderedDict
from dadata import DaDataClient

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg' 

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [ ]:
# Путь к базе Альфы.
database_file_path = ["data/alpha_base/ЛПУ_АВИС_20171012.xlsx"]

# Путь к запросу.
client_file_path = [
    "data/request/Размеченное_1.xlsx"
]

# npz файлы базы и запроса.
alpha_npz_file = "data/alpha_base/dadata_alpha_npz_file_20171012.npz"
request_npz_file = "data/request/dadata_uniq.npz"

# Позиции адреса и наименования в базе.
addr_cols_1 = [[6]]
name_cols_1 = [[4]]

# Позиции адреса и наименования в запросе.
addr_cols_2 = [[0]]
name_cols_2 = [[1]]

# Настройка для для работы с n-граммами.
use_components = ['name', 'addr']
type_options = ['char', 'word']
n_char = [3]
n_word = [1]

# Топ N, возвращаемый n-граммами.
top_n = [5]

# Тип работы с n-граммами.
algoritm_types = ['weighed', 'not_weighed']

# Тип сервиса.
api_types = ['dadata']

# Id из размеченного альфой файла.
uniq_df = pd.read_excel("data/request/Размеченное_1.xlsx")
uniq_id = uniq_df['id'].values

template = ['ё']
change_on = ['е']

# Ключ dadata
key = '00ce663fcf3e469ca0714b4d5eab82a4c0ccc2df'
secret = '' 

# Прочее
my_list = []

In [ ]:
%%time

index_1 = LpuIndex(database_file_path, addr_cols_1, name_cols_1).fill_indices()
index_2 = LpuIndex(client_file_path, addr_cols_2, name_cols_2).fill_indices()

A = (LpuList(
    index_1
).load(fmt='npz', npz_file_path=alpha_npz_file)
 .dadata_request(key=key, secret=secret)
 .to_tree()
 .address_features_extaction()
 .change_symb(template=template, change_on=change_on)
 .clear_name_and_address()
 .get_token_list(use_components=['name', 'addr'])
 .w2v(path_to_model="data/w2v.bin", use_components=['name', 'addr']))
    
B = (LpuList(
    index_2
).load(fmt='npz', npz_file_path=request_npz_file)           
 .dadata_request(key=key, secret=secret)
 .address_features_extaction()
 .change_symb(template=template, change_on=change_on)
 .clear_name_and_address()
 .get_token_list(use_components=['name', 'addr'])
 .w2v(path_to_model="data/w2v.bin", use_components=['name', 'addr'])) 
       
A.prepare_min_hash_and_lsh(B, algoritm_types=algoritm_types, type_options=type_options,
                           n_char=n_char, n_word=n_word, use_components=use_components)

In [ ]:
%time

T = (Train(A, B, k=50)
 .prepare_candidates(n=50, options=[
     ('weighed', 'word', 1),
     ('not_weighed', 'word', 1),
     ('not_weighed', 'char', 3),
 ]
).create_model(answer_file_path='data/request/Размеченное_1.xlsx')
    )

In [ ]:
all_finded = T._results_on_metrics('data/request/Размеченное_1.xlsx', 50) 
top_1 = T._results_on_metrics('data/request/Размеченное_1.xlsx', 1)
top_5 = T._results_on_metrics('data/request/Размеченное_1.xlsx', 5)
top_10 = T._results_on_metrics('data/request/Размеченное_1.xlsx', 10)

In [ ]:
print("Метрика топ-1: ", top_1 / all_finded)
print("Метрика топ-5: ", top_5 / all_finded)
print("Метрика топ-10: ", top_10 / all_finded)